# Methylation Evaluation

The [minfi](https://htmlpreview.github.io/?https://github.com/hansenlab/tutorial.450k/blob/master/inst/doc/methylation450k.html) package is commonly used for methylation array analysis. Normalization is handled by the included subset-quantile within array normalization (SWAN) method. SWAN normalizes methylation array data by selecting a subset of probes from both types (Infinium I and Infinium II).

The SWAN publication describes this method thusly:

> The SWAN method has two parts. First, an average quantile distribution is determined using a subset of probes defined to be biologically similar based on CpG content. This is achieved by randomly selecting N Infinium I and II probes that have one, two and three underlying CpGs, where N is the minimum number of probes in the six sets of Infinium I and II probes with one, two and three probe body CpGs. If no probes have been filtered out - for example, sex chromosome probes, and so on - N = 11,303. This results in a pool of 3N Infinium I and 3N Infinium II probes. Due to the vast differences in their distributions (Figure 2), the subsequent processing is performed independently on both the methylated and unmethylated channels. The subset for each probe type, from each channel (methylated or unmethylated), is sorted by increasing intensity. The value of each of the 3N pairs of observations is then assigned to be the mean intensity of the two probe types for that row or 'quantile'. This is the standard quantile procedure. The second step is to then adjust the intensities of the remaining probes, of which there are many more Infinium II than I, by interpolation onto the distribution of the subset probes. This is done for each probe type separately using linear interpolation between the subset probes to define the new intensities. Consequently, while the distribution of the subset is identical, the intensity distribution of Infinium I probes is still vastly different from the distribution of Infinium II probes (Figure S2 in Additional file 1).

This description suggests that individual samples can be normalized separately since the distribution of each channel within a sample is adjusted based on the subset of probes selected. The only difference between independent runs of SWAN is the selection of the subset of probes. This can be controlled by setting a seed in R (`set.seed(N)`).


# Setup
This evaluation requires a number of packages to be installed in the environment. To accomplish this, I have used [conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html) along with the `bioconda`, `r`, and `conda-forge` channels. 

In [1]:
%%bash
# Uncomment the lines below to create the conda environment.
# - I'm not necessarily advocating you do this in this notebook, 
#   probably want to set it up in a terminal and then start this Jupyter notebook after :).

# conda create -n methylation \
#              -c anaconda \
#              -c bioconda \
#              -c conda-forge \
#              numpy \
#              pandas \
#              r-dplyr \
#              bioconductor-minfi \
#              bioconductor-illuminahumanmethylationepicmanifest \
#              bioconductor-illuminahumanmethylationepicanno.ilm10b4.hg19 \
#              
#              -y
# conda init bash
# source ~/.bashrc
# conda activate methylation

# Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
R.version.string

[1] "R version 4.2.2 (2022-10-31)"


In [5]:
%%R
#Load libraries
library(dplyr)
library(minfi)
library(IlluminaHumanMethylationEPICmanifest)
library(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sap

For reproducibility, we'll fix the seed value.

In [6]:
%%R
set.seed(1)

In [7]:
%%R
RGSet <- read.metharray("~/data/COMET/methylation_data/201533640034_R01C01", verbose = TRUE, force = TRUE)

R[write to console]: [read.metharray] Reading 201533640034_R01C01_Grn.idat

R[write to console]: [read.metharray] Reading 201533640034_R01C01_Red.idat

R[write to console]: [read.metharray] Read idat files in 0.6 seconds

R[write to console]: [read.metharray] Creating data matrices ... 
R[write to console]: done in 1.4 seconds

R[write to console]: [read.metharray] Instantiating final object ... 
R[write to console]: done in 0.0 seconds



In [8]:
%%R
manifest <- getManifest(RGSet)
manifest

IlluminaMethylationManifest object
Annotation
  array: IlluminaHumanMethylationEPIC
Number of type I probes: 142262 
Number of type II probes: 724574 
Number of control probes: 635 
Number of SNP type I probes: 21 
Number of SNP type II probes: 38 


In [9]:
%%R
# Load raw data into a MethylSet object be converting red/green
# channels into a matrix of methlyated and unmethylated signals
MSet <- preprocessRaw(RGSet)

In [10]:
%%R
# Convert to a RatioSet
RSet <- ratioConvert(MSet, what = "both", keepCN = TRUE)

In [11]:
%%R
# Add genomic coordinates to each probe (plus additional annotation)
GRset <- mapToGenome(RSet)

In [12]:
%%R
# Take the genomic mapped RatioSet and fill Beta values (non-normalized).
# Get the NON-normalized beta values:
beta <- getBeta(GRset)

In [13]:
%%R
# Get M-value matrix and copy-number matrix
M <- getM(GRset)
CN <- getCN(GRset)

In [14]:
%%R
# Get sample names
sampleNames <- sampleNames(GRset)
sampleNames

[1] "201533640034_R01C01"


In [15]:
%%R
# Get probe names
featureNames <- featureNames(GRset)
print(head(featureNames))
length(featureNames)

[1] "cg14817997" "cg26928153" "cg16269199" "cg13869341" "cg14008030"
[6] "cg12045430"
[1] 865859


In [16]:
%%R
gr <- granges(GRset)
annotation <- getAnnotation(GRset)

In [17]:
%%R
set.seed(1)
# Perform SWAN normalization on beta values
GRset.swan_norm <- preprocessSWAN(RGSet)
beta_swan_norm <- getBeta(GRset.swan_norm)

In [18]:
%%R -o beta_swan_norm_genomic
# Write the normalized beta-values that have NOT yet had
# low-variance probes filtered out
# Filter to only those that are mappable to the genome.
RSet_genomic <- ratioConvert(GRset.swan_norm)
GRset_genomic <- mapToGenome(RSet_genomic)
beta_swan_norm_genomic <- getBeta(GRset_genomic)
colnames(beta_swan_norm_genomic) <- colnames(CN)

In [19]:
print(beta_swan_norm_genomic)
print(beta_swan_norm_genomic.shape)

[[0.8319394 ]
 [0.87825323]
 [0.74041298]
 ...
 [0.84366217]
 [0.5464304 ]
 [0.41737797]]
(865859, 1)


Read in data for a second sample.

In [20]:
%%R
second_RGSet <- read.metharray("~/data/COMET/methylation_data/201533640009_R08C01", verbose = TRUE, force = TRUE)

R[write to console]: [read.metharray] Reading 201533640009_R08C01_Grn.idat

R[write to console]: [read.metharray] Reading 201533640009_R08C01_Red.idat

R[write to console]: [read.metharray] Read idat files in 0.9 seconds

R[write to console]: [read.metharray] Creating data matrices ... 
R[write to console]: done in 1.1 seconds

R[write to console]: [read.metharray] Instantiating final object ... 
R[write to console]: done in 0.0 seconds



Now we'll combine the data for both samples so that we can analysis them together.

In [21]:
%%R
combined <- combine(RGSet, second_RGSet)

In [22]:
%%R -o combined_beta_swan_norm_genomic
set.seed(1)
combined_GRset <- mapToGenome(combined)
combined_CN <- getCN(combined_GRset)
combined_GRset.swan_norm <- preprocessSWAN(combined)
combined_RSet_genomic <- ratioConvert(combined_GRset.swan_norm)
combined_GRset_genomic <- mapToGenome(combined_RSet_genomic)
combined_beta_swan_norm_genomic <- getBeta(combined_GRset_genomic)
colnames(combined_beta_swan_norm_genomic) <- colnames(combined_CN)


In [23]:
print(combined_beta_swan_norm_genomic)

[[0.8319394  0.91933784]
 [0.87825323 0.86565846]
 [0.74041298 0.74228972]
 ...
 [0.84366217 0.81228846]
 [0.5464304  0.49253039]
 [0.41737797 0.17487666]]


In [24]:
sample_from_combined = combined_beta_swan_norm_genomic[:,0]
print(sample_from_combined)

[0.8319394  0.87825323 0.74041298 ... 0.84366217 0.5464304  0.41737797]


Now to check equality between the two runs of the normalization.

In [25]:
np.array_equal(beta_swan_norm_genomic, sample_from_combined)

False

The inequality is not a surprise. These are very long floating point numbers, so some difference is expected due to rounding. So we need to compare with a tolerance.

Numpy offers the `allclose` function which compares array values within a tolerance. However, it crashes this kernel, presumably, due to the size of the arrays.

In [26]:
#np.allclose(beta_swan_norm_genomic, sample_from_combined)

Instead we will iterate over the arrays and check manually. Then check to see that all values are within the allowable tolerance.

In [27]:
value = []
atol = 1e-8
rtol = 1e-5
for a,b in zip(beta_swan_norm_genomic, sample_from_combined):
    value.append(abs(a - b) <= (atol + rtol * abs(b)))

In [28]:
print(all(value))

True
